Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 02 - Neuronale Netze mit Tensorflow

Nachdem wir uns gerade angeguckt haben, was Tensorflow grundlegen ist und das Arbeiten mit Daten aufgefrischt haben, wollen wir uns als Nächstes den Grundlagen der Neuronalen Netze widmen.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
print(tf.__version__)

## Neuronen und Layer

Zuerst schauen wir uns dabei ein Neuron sowie den Aufbau eines Layers an. Wobei ein Layer aus einen, bis mehreren Neuronen besteht.
Ein Neuron besteht dabei aus drei Funktionen:
* Übertragungsfunktion: Summe alle gewichteten Inputs addiert um den Bias-Wert
* Aktivierungsfunktion: z. B. ReLU
* Ausgabefunktion: Ausgabe Funktion in der Regel Identität

### lineare Layer
Jedes neuronale Netz besteht aus mehreren Schichten. Die Schichten sind also die Bausteine. 
Unsere erste Layer ist eine lineare Layer, die nur den Drive berechntet. Wir verwenden hier keine Aktivierungsfunktion.

Um eine Layer zu definieren, brauchen wir eine Klasse, die von `tf.keras.layers.Layer` erbt.
Zusätzlich enthält TensorFlow 2 bereits viele eingebaute Schichten, die Ihr hier finden könnt: https://www.tensorflow.org/api_docs/python/tf/keras/layers.

In [ ]:
from keras.layers import Layer

class Linear(Layer):
    
    def __init__(self, units):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(
                        shape=(input_shape[-1], self.units),
                        initializer=tf.random_normal_initializer(),
                        trainable=True
        )
        self.b = self.add_weight(
                        shape=(self.units,),
                        initializer=tf.random_normal_initializer(),
                        trainable=True
        )
     
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

Test des linearen Layers

In [ ]:
linear_layer = Linear(4)

x = tf.ones((1,4))
y = linear_layer(x)

print(x)
print(y)

### Mehrere Layer verbinden
Um mehrere Layer miteinander zu verknüpfen, können wir diese in einer neuen Klasse, die wiederum ein Layer ist kombinieren.
Dabei sprechen wir von einem Dens Layer, wenn alle Neuronen aus der vorherigen Layer mit allen Neuronen des aktuellen Layers verknüpft sind.

Arten von Layern
* Input: Erste schicht im Netz
* Hidden: Zwischen schichten im Netz
* Output: Ausgabe Schicht im Netz

Hier findet ihr eine anschauliche [Visualisierung](https://playground.tensorflow.org) des Prinzipes.

In [ ]:
class MLP(Layer):
    
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden_layer = Linear(512)
        self.output_layer = Linear(1)
        
    def call(self, x):
        x = self.hidden_layer(x)
        x = tf.nn.relu(x)
        x = self.output_layer(x)
        return x

### Training des neuronalen Netzes
Um das neuronale Netz zu trainieren, müssen wir zuerst noch die Loss-Funktion ([Dokumentation](https://www.tensorflow.org/api_docs/python/tf/keras/losses)) und den Optimierer ([Dokumentation](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)) bestimmen.

In [ ]:
tf.keras.backend.clear_session()

mlp = MLP()

mse = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

Zum Training verwenden wir den Datensatz aus dem ersten Kapitel

In [ ]:
def f(x):
    return 0.002*(x**3-x**2+2*x)+0.3

xs = np.linspace(-5,5, 20, dtype=np.float32)
ys = f(xs)

ids = np.arange(len(xs))
training_data_ids = np.random.choice(ids,15, replace=False)
test_data_ids = ~np.isin(ids, training_data_ids)
training_data_xs = xs[training_data_ids]
training_data_ys = ys[training_data_ids]
test_data_xs = xs[test_data_ids]
test_data_ys = ys[test_data_ids]

train_dataset = tf.data.Dataset.from_tensor_slices((training_data_xs, training_data_ys)).batch(15)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data_xs, test_data_ys)).batch(5)

Dann müssen wir nur noch unsere Trainingsschleife bauen.

In [ ]:
train_losses = []
test_losses = []
epochs = []

for epoch in range(1000):
    epochs.append(epoch)
    
    for (x,y) in train_dataset:
        
        x = tf.reshape(x, shape=(-1,1))

        with tf.GradientTape() as tape:
            output = mlp(x)
            loss = mse(y, output)
            train_losses.append(loss)
            gradients = tape.gradient(loss, mlp.trainable_variables)

        optimizer.apply_gradients(zip(gradients, mlp.trainable_variables))        

    for (x,y) in test_dataset:
        x = tf.reshape(x, shape=(-1,1))
        output = mlp(x)
        loss = mse(y, output)       
        test_losses.append(loss)

Anschließend können wir mit unseren gesammelten Werten den Traingsprozess visualisieren.

In [ ]:
plt.figure()
plt.plot(epochs, train_losses)
plt.plot(epochs, np.array(test_losses))
plt.legend(("train","test"))
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.show()

Und zum Schluss können wir auch noch die Approximation visualisieren. 

In [ ]:
plt.figure()
plt.scatter(training_data_xs, training_data_ys, c='red')
plt.scatter(test_data_xs, test_data_ys, c='blue')

xs = np.linspace(-5,5,100, dtype=np.float32)
xs = np.reshape(xs, newshape=(-1,1))
ys = mlp(xs)

plt.plot(xs,ys)
plt.xlim(-5,5)
plt.ylim(-0.05,0.6)
plt.show()

## Multilayer perceptron (MLP)

### Aktivierungsfunktionen
Wie erwähnt gibt es verschiedene Aktivierungsfunktionen. Diese kommen je nach Aufgabe oder Layer zum Einsatz:
- Layer
  - Hidden: Bei MLPs kommt in der Regel die im Hidden Layer eine ReLU zum Einsatz (es können aber auch andere verwandte Funktionen versucht werden wie: Leaky ReLU / Parametric ReLU, ELU oder SeLU)
  - Output: 
    - Regression: Hier wird häufig eine lineare Funktion bzw die Identität verwendet 
    - Classification:
      - Binary oder Multi-Label Classification: Also Funktion sollte im Output Sigmoid verwendet werden
      - Multi-Class Classification: Also Funktion sollte im Output Softmax verwendet werden

Zusätzlich sollte erwähnt werden das es noch die TanH Funktion gibt. Zum nachlesen findet ihr [hier](https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/) mehr.


### Gewichts Initialisierung
Um Probleme beim Lernen zu vermeiden kann es sinnvoll sein die lernbaren gewichte mit Werten zu initialisieren. Ein Beispiel hierfür ist das Verwenden von He Initialisierung der Gewichte und Biases für ReLU-Aktivierungsfunktion. Weitere Initialisierung lassen sich im [hier](https://www.tensorflow.org/api_docs/python/tf/keras/initializers) und im [Paper](https://arxiv.org/ftp/arxiv/papers/2102/2102.07004.pdf) finden.

In der nächsten Zelle erstellen wir das neuronale Netz. Dafür verwenden wir ein [sequentielles Modell](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential?hl=en). Diese hilft dabei ein Modell aus einer Liste von Layern zu erstellen und die Layer zu trainieren. Als Layer nehmen wir das [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense?hl=en)-Layer.

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

iris = datasets.load_iris()
y = to_categorical(iris.target)

x_train, x_test, y_train, y_test = train_test_split(iris.data, y, test_size=0.3, random_state=42)

In [ ]:
from keras import Sequential
from keras.layers import Dense
from keras.initializers import HeNormal

model = Sequential(name="classification_mlp")
model.add(Dense(8, activation='relu', input_dim=4, bias_initializer=HeNormal(), kernel_initializer=HeNormal()))
model.add(Dense(3, activation='softmax'))

Im Folgenden wir gezeigt, wie wir die Struktur unseres neuronalen Netzes ausgeben lassen können.

In [ ]:
model.summary()

In [ ]:
from keras import utils

utils.plot_model(
    model, 
    show_shapes=True, 
    show_dtype=False,
    show_layer_names=True,
    show_layer_activations=True
)

## Loss, Optimizer und Metriken

Folgend sind die Dokumentationen für die Loss-Funktionen, Optimiere und Metriken verlinkt. Als kleine Empfehlung könnt ihr meistens gut auf adaptiven Optimierer zurückgreifen wie `Adam`.
* [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses)
* [Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
* [Metriken](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [ ]:
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import (
    CategoricalAccuracy,
    Recall,
    Precision,
)


model.compile(
    optimizer=Adam(learning_rate=0.1),
    loss=CategoricalCrossentropy(),
    metrics=[
        CategoricalAccuracy(),
        Recall(),
        Precision(),
    ]
)

## Einrichten von TensorBoard
Um die Visualisierung des Trainings zu automatisieren, kann [TensorBoard](https://www.tensorflow.org/tensorboard) verwendet werden. Dieses hilft dabei den Trainingsfortschritt zu visualisieren. Um TensorBoard im Training zu aktivieren müssen wir uns eine callback-Funktion erstellen ([Doku](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard?hl=en)). 

In [ ]:
from datetime import datetime
from keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(
    log_dir=f"logs/fit/{datetime.now().strftime('%Y%m%d-%H%M%S')}", 
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
)

## Trainieren des neuronalen Netzes
Für das Training müssen wir nicht unbedingt eine Trainingschleife schreiben. Stattdessen können wir auch die `fit()`-Methode des `Sequential`-Modells verwenden.
Wenn der Loss sich beim Lernen komisch verhält, ist das ein Anzeichen, das noch nicht optimal gelernt wird. Weiter Infos sind [hier](https://developers.google.com/machine-learning/testing-debugging/metrics/interpretic?hl=de) zu finden.

In [ ]:
model.fit(
    x=x_train, 
    y=y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=25,
    callbacks=[tensorboard_callback]
)

## Evaluation

In [ ]:
loss, acc, rec, prec = model.evaluate(x_test, y_test)

In [ ]:
%load_ext tensorboard
%tensorboard --host 127.0.0.1 --port=8080 --logdir logs/fit

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)

y_test_classes = np.argmax(y_test, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test_classes, y_pred_classes))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_test_classes, y_pred_classes, display_labels=iris.target_names)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, threshold = roc_curve(y_test.ravel(), y_pred.ravel())
    
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc(fpr, tpr)))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend()

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 